# Data preprocessing

The code in this notebook takes the input pictures and applies the mediapipe hands solution to find the relevant coordinates and saves them as a .csv file which can be used by the AI model to train.

## Imports

In [1]:
!pip install mediapipe

import pandas as pd
import cv2
import mediapipe as mp
import pandas as pd

     |████████████████████████████████| 32.8 MB 52 kB/s 


## Download database

In [8]:
!pip install kaggle
# kaggle API
!echo '{"username":"spacewaiker","key":"062bca10262b87f5b8498c589cdc65c6"}' > ~/.kaggle/kaggle.json

In [9]:
# download dataset:
!kaggle datasets download grassknoted/asl-alphabet
# extract files (-q disables writing status message for every file)
!unzip -q asl-alphabet.zip
!ls

 99% 1.01G/1.03G [00:08<00:00, 163MB/s]
100% 1.03G/1.03G [00:08<00:00, 132MB/s]
asl_alphabet_test  asl_alphabet_train  asl-alphabet.zip  drive	sample_data


Symbols:

|Command  |ASL letters|Sign           |
|---------|-----------|---------------|
|Line     |L          |"L" shape      |
|Ellipse  |A, S       |Fist           |
|Rectangle|G          |Point right    |
|Triangle |K, V       |Two fingers up |
|Move     |B          |Palm           |
|Delete   |R          |Crossed fingers|

## Define functions to use later

In [10]:
'''
Function to return the file names with the following format:
A/A1234.jpg     if letter = 'A'  and   i = 1234
'''
def get_file_names(letter, begin, end):
    files = []
    for i in range(begin, end+1):
        files.append(f'{letter}/{letter}{i}.jpg')
    
    return files

'''
Function that applies the mediapipe hands solution to an image.
img is the image file name and hands is the hands instance to use
'''
def apply_hands(row):
    global hands
    img = row.name
    result = row.copy()

    image = cv2.imread(DIR + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    landmk = hands.process(image).multi_hand_world_landmarks

    if landmk:    # success
        landmarks = landmk[0]
        list_tuples = [(i.x, i.y, i.z) for i in landmarks.landmark]
        result[1:64] = [i for t in list_tuples for i in t]

    return result

In [11]:
# Mediapipe stuff
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get file names
# only select the first 2900 files, as to leave 100 for the testing dataset
file_names = pd.DataFrame()

file_names = file_names.append([['Line',      file] for file in get_file_names('L', 1, 2900)])
file_names = file_names.append([['Ellipse',   file] for file in get_file_names('A', 1, 1450) + get_file_names('S', 1, 1450)])
file_names = file_names.append([['Rectangle', file] for file in get_file_names('G', 1, 2900)])
file_names = file_names.append([['Triangle',  file] for file in get_file_names('K', 1, 1450) + get_file_names('V', 1, 1450)])
file_names = file_names.append([['Move',      file] for file in get_file_names('B', 1, 2900)])
file_names = file_names.append([['Delete',    file] for file in get_file_names('R', 1, 2900)])

file_names.rename(columns={0: 'Command', 1: 'File'}, inplace=True)
file_names.set_index('File', inplace=True)
print(file_names)

            Command
File               
L/L1.jpg       Line
L/L2.jpg       Line
L/L3.jpg       Line
L/L4.jpg       Line
L/L5.jpg       Line
...             ...
R/R2896.jpg  Delete
R/R2897.jpg  Delete
R/R2898.jpg  Delete
R/R2899.jpg  Delete
R/R2900.jpg  Delete

[17400 rows x 1 columns]


In [12]:
DIR = 'asl_alphabet_train/asl_alphabet_train/'

cols = ['Command'] + [i for i in range(21*3)]
train_x = pd.DataFrame(columns=cols, index=file_names.index)

train_x['Command'] = file_names['Command']


# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    train_x = train_x.apply(apply_hands, axis=1)

print(train_x)

            Command         0         1  ...        60        61        62
File                                     ...                              
L/L1.jpg       Line -0.029041  0.077856  ... -0.037617  0.031186 -0.024797
L/L2.jpg       Line -0.030712  0.077171  ... -0.036257  0.028472 -0.026228
L/L3.jpg       Line -0.032309  0.073458  ... -0.034835  0.028049 -0.026690
L/L4.jpg       Line -0.032427  0.076893  ... -0.034228  0.028305 -0.025699
L/L5.jpg       Line -0.034163  0.076618  ... -0.035511  0.031026 -0.027471
...             ...       ...       ...  ...       ...       ...       ...
R/R2896.jpg  Delete  0.013954  0.085491  ... -0.012614  0.030631 -0.026084
R/R2897.jpg  Delete  0.003139  0.089351  ... -0.012169  0.025248 -0.029481
R/R2898.jpg  Delete  0.010776  0.091557  ... -0.008634  0.037975 -0.027085
R/R2899.jpg  Delete  0.011940  0.089052  ... -0.011437  0.030680 -0.026755
R/R2900.jpg  Delete  0.012687  0.089746  ... -0.010888  0.034238 -0.025730

[17400 rows x 64 columns

In [13]:
train_x.to_csv("drive/MyDrive/MAISFinalProject/train_x.csv")

## Preprocess testing data

In [19]:
# get file names
# select the last 100 images
file_train = pd.DataFrame()

file_train = file_train.append([['Line',      file] for file in get_file_names('L', 2901, 3000)])
file_train = file_train.append([['Ellipse',   file] for file in get_file_names('A', 2901, 2950) + get_file_names('S', 2901, 2950)])
file_train = file_train.append([['Rectangle', file] for file in get_file_names('G', 2901, 3000)])
file_train = file_train.append([['Triangle',  file] for file in get_file_names('K', 2901, 2950) + get_file_names('A', 2901, 2950)])
file_train = file_train.append([['Move',      file] for file in get_file_names('B', 2901, 3000)])
file_train = file_train.append([['Delete',    file] for file in get_file_names('R', 2901, 3000)])

file_train.rename(columns={0: 'Command', 1: 'File'}, inplace=True)
file_train.set_index('File', inplace=True)

test_x = pd.DataFrame(columns=cols, index=file_train.index)

test_x['Command'] = file_train['Command']

# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    test_x = test_x.apply(apply_hands, axis=1)

print(test_x)

            Command         0         1  ...        60        61        62
File                                     ...                              
L/L2901.jpg    Line -0.026775  0.082416  ... -0.030209  0.019278 -0.036167
L/L2902.jpg    Line -0.029573  0.081979  ... -0.029755  0.019623 -0.036395
L/L2903.jpg    Line -0.028724  0.082270  ... -0.030055  0.018456 -0.035738
L/L2904.jpg    Line -0.027211  0.083636  ... -0.029574  0.018555 -0.036874
L/L2905.jpg    Line -0.029691  0.082440  ... -0.029960  0.018194 -0.035498
...             ...       ...       ...  ...       ...       ...       ...
R/R2996.jpg  Delete -0.007209  0.092140  ... -0.008528  0.031733 -0.034578
R/R2997.jpg  Delete -0.006988  0.091541  ... -0.007468  0.032481 -0.033076
R/R2998.jpg  Delete -0.009442  0.092719  ... -0.010438  0.034792 -0.031444
R/R2999.jpg  Delete -0.006774  0.092433  ... -0.012130  0.033123 -0.031096
R/R3000.jpg  Delete -0.007207  0.091573  ... -0.012152  0.030552 -0.032354

[600 rows x 64 columns]


In [20]:
test_x.to_csv('drive/MyDrive/MAISFinalProject/test_x.csv')